<a href="https://colab.research.google.com/github/Shubhamgupta171/6Companies30DayChallenge/blob/main/Hindi_OCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import os

# Define paths
Users_dir=os.path.expanduser('~/Users')
shubhamgupta_dir=os.path.join(Users_dir,'shubhamgupta')
downloads_dir = os.path.join('~/Downloads')
dataset_dir = os.path.join(downloads_dir, 'devanagari')
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

# Print the directory structure to verify
print(f"Users directory: {Users_dir}")
print(f"ShubhamGupta directory: {shubhamgupta_dir}")
print(f"Downloads directory: {downloads_dir}")
print(f"Dataset directory: {dataset_dir}")
print(f"Train directory: {train_dir}")
print(f"Test directory: {test_dir}")

# List contents of the directories
print("Contents of Users directory:")
print(os.listdir(Users_dir))

print("Contents of ShubhamGupta directory:")
print(os.listdir(shubhamgupta_dir))

print("Contents of Downloads directory:")
print(os.listdir(downloads_dir))

if os.path.exists(dataset_dir):
    print("\nContents of Devanagari dataset directory:")
    print(os.listdir(dataset_dir))

if os.path.exists(train_dir):
    print("\nContents of Train directory:")
    print(os.listdir(train_dir))

if os.path.exists(test_dir):
    print("\nContents of Test directory:")
    print(os.listdir(test_dir))


Users directory: /root/Users
ShubhamGupta directory: /root/Users/shubhamgupta
Downloads directory: ~/Downloads
Dataset directory: ~/Downloads/devanagari
Train directory: ~/Downloads/devanagari/train
Test directory: ~/Downloads/devanagari/test
Contents of Users directory:


FileNotFoundError: [Errno 2] No such file or directory: '/root/Users'

In [23]:
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import requests
from PIL import Image
from io import BytesIO

# Define paths
train_dir = os.path.expanduser('~/Users/shubhamgupta/Downloads/devanagari/train')
test_dir = os.path.expanduser('~/Users/shubhamgupta/Downloads/devanagari/test')

# Check if the directories exist
if not os.path.exists(train_dir):
    raise FileNotFoundError(f"Training directory '{train_dir}' not found.")
if not os.path.exists(test_dir):
    raise FileNotFoundError(f"Testing directory '{test_dir}' not found.")

# Define the CNN model
def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Data augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False
)

test_datagen = ImageDataGenerator(rescale=1.0/255)

# Data generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(32, 32),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(32, 32),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

# Get the input shape and number of classes
input_shape = (32, 32, 1)
num_classes = len(train_generator.class_indices)

# Create the model
model = create_model(input_shape, num_classes)

# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator,
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=20
)

# Save the model
model.save('hindi_ocr_cnn.h5')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test accuracy: {accuracy * 100:.2f}%")

# Function to preprocess and predict the character from an image
def predict_character(image_path, model):
    # Download the image from URL
    response = requests.get(image_path)
    image = Image.open(BytesIO(response.content)).convert('L')
    image = image.resize((32, 32))
    image = np.array(image).astype('float32') / 255
    image = np.expand_dims(image, axis=-1)
    image = np.expand_dims(image, axis=0)

    prediction = model.predict(image)
    predicted_class = np.argmax(prediction)

    return predicted_class

# Example usage
# Replace 'path_to_hindi_character_image' with the actual path to an image file
image_path = 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQTIdVT6r94egLNXVbfyty9FManGBDe-9JrEhHMRMc6OR3nGz7vwF4k1s--G4Fr0-ns7hI&usqp=CAU'
predicted_class = predict_character(image_path, model)
print(f"Predicted class: {predicted_class}")


FileNotFoundError: Training directory '/root/Users/shubhamgupta/Downloads/devanagari/train' not found.